# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d200/7037ec05b97576da1bd9db5e3ea6a322a577bc833b0dc772238248ef.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Toronto, Ontario, Canada**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(200,'7037ec05b97576da1bd9db5e3ea6a322a577bc833b0dc772238248ef')

# Data Processing

In [ ]:
import pandas as pd

# read csv
weather_df = pd.read_csv('data/C2A2_data/BinnedCsvs_d200/7037ec05b97576da1bd9db5e3ea6a322a577bc833b0dc772238248ef.csv')
weather_df.to_csv('Assignment2_raw_CentralOntarioWeather.csv')

# process weather data - drop id, sort by date
weather_df = weather_df.drop(['ID'], axis=1).sort_values(by='Date')#.set_index('Date')
# compute lowest daily tempareture across the region
minTemp_df = weather_df[weather_df['Element']=='TMIN'].drop(['Element'], axis=1)
minTemp_df = minTemp_df.groupby('Date').agg({"Data_Value":[min]})
minTemp_df.columns = ['TMIN']
# compute highest daily tempareture across the region
maxTemp_df = weather_df[weather_df['Element']=='TMAX'].drop(['Element'], axis=1)
maxTemp_df = maxTemp_df.groupby('Date').agg({"Data_Value":[max]})
maxTemp_df.columns = ['TMAX']

# temperature df by date
temperature_df = pd.concat([minTemp_df,maxTemp_df], axis=1, join='inner').reset_index()

#import datetime
# convert string to timestamp
temperature_df['Date'] = pd.to_datetime(temperature_df['Date'])#.dt.date #, format='%d%b%Y:%H:%M:%S.%f')
type(temperature_df['Date'].iloc[0])
temperature_df['Year'] = temperature_df['Date'].dt.strftime('%Y')
temperature_df['MonthDay'] = temperature_df['Date'].dt.strftime('%m-%d')
temperature_df['DayOfYear'] = temperature_df['Date'].dt.strftime('%j').map(int)

# remove leap day: delete month day of 02-29
temperature_df = temperature_df[(temperature_df['MonthDay']!='02-29')]

# save processed data to csv
temperature_df.to_csv('Assignment2_proc_temperature.csv', index=False)

In [ ]:
# get final temperature by day dataframe
temperatureByDay_df = temperature_df.groupby('MonthDay').agg({"TMIN":[min], "TMAX":[max],}).reset_index()
temperatureByDay_df.columns = ['MonthDay', 'TMIN', 'TMAX']
temperatureByDay_df['DayOfYear'] = temperatureByDay_df.index + 1

# save processed data to csv
temperatureByDay_df.to_csv('Assignment2_proc_temperatureByDay.csv', index=False)

## Load Processed Data

In [2]:
import pandas as pd

temperatureByDay_df = pd.read_csv('Assignment2_proc_temperatureByDay.csv')
temperatureByDay_df

MonthDay  TMIN  TMAX
0      01-01  -250   134
1      01-02  -245   139
2      01-03  -270   161
3      01-04  -222   144
4      01-05  -200   150
5      01-06  -260   160
6      01-07  -270   178
7      01-08  -260   194
8      01-09  -236   183
9      01-10  -229   110
10     01-11  -200   150
11     01-12  -250   161
12     01-13  -260   206
13     01-14  -290   200
14     01-15  -290    89
15     01-16  -260   103
16     01-17  -270   128
17     01-18  -275   128
18     01-19  -228   117
19     01-20  -250   136
20     01-21  -320   139
21     01-22  -306   117
22     01-23  -280   106
23     01-24  -295   106
24     01-25  -272   106
25     01-26  -260    89
26     01-27  -311    83
27     01-28  -300   125
28     01-29  -260   170
29     01-30  -255   189
..       ...   ...   ...
335    12-02  -135   140
336    12-03  -114   170
337    12-04  -130   191
338    12-05  -159   194
339    12-06  -180   167
340    12-07  -213   100
341    12-08  -214   120
342    12-09  -210   144
343    12-10  -185   150
344    12-11  -175   156
345    12-12  -220   161
346    12-13  -220   200
347    12-14  -206   222
348    12-15  -189   222
349    12-16  -230   144
350    12-17  -245   150
351    12-18  -167   128
352    12-19  -159    83
353    12-20  -230   100
354    12-21  -190   144
355    12-22  -180   167
356    12-23  -170   183
357    12-24  -230   189
358    12-25  -224   172
359    12-26  -194   122
360    12-27  -200   178
361    12-28  -160   178
362    12-29  -180   172
363    12-30  -170   100
364    12-31  -170   133

[365 rows x 3 columns]

# Plot Functions for time series data

In [5]:
# don't forget the python magic for interactive matplotlib environment

%matplotlib notebook

import matplotlib as mpl
import matplotlib.pyplot as plt

In [12]:
def simpleLinePlot(data, y, X, ylabel, xlabel, lineTyp='-'):
    
    """Construct a simple line plot with provided data, 
    series definition, and labels.
    data: dataframe object
    y: name of the y
    X: 
    """
    
    import matplotlib.pyplot as plt
    import numpy as np
    
    plt.figure()
    
    # plot data
    for x in X:
        plt.plot(data[X], lineTyp)    

    plt.show()
    

In [3]:
simpleLinePlot(temperatureByDay_df)

In [13]:
simpleLinePlot?